In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## Introduction

The purpose of this notebook is to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed, based on their historical preferences.



## Installed packages


In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import Reader
from surprise import Dataset

from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy


## Read Data


In [8]:
df_sample_submission = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/sample_submission.csv')
df_movies = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv')
df_imdb = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/imdb_data.csv')
df_genome_scores = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/genome_scores.csv')
df_genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/genome_tags.csv')
df_train = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/train.csv')
df_test = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv')
df_tags = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/tags.csv')
df_links = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/links.csv')


## Explorer


In [13]:
#Lets check out the genres:

genres = pd.DataFrame(df_movies[['movieId', 'genres']], columns=['movieId', 'genres'])
genres.genres = genres.genres.apply(lambda x: x.split('|'))
genres = pd.DataFrame([(tup.movieId, d) for tup in genres.itertuples() for d in tup.genres], columns=['movieId', 'genres'])

plot = plt.figure(figsize=(15, 10))
plt.title('genres\n', fontsize=20)
sns.countplot(y="genres", data=genres,order=genres['genres'].value_counts(ascending=False).index, palette='Pastel1')
plt.show()


## Modelling


In [20]:
data = Dataset.load_from_df(df_train[['userId', 'movieId', 'rating']], Reader())

In [21]:
train_set, test_set = train_test_split(data, test_size=0.20)

In [22]:
svd=SVD(n_epochs = 30, n_factors = 200, init_std_dev = 0.05, random_state=42)
svd.fit(train_set)

In [25]:
test_pred= svd.test(test_set)

In [26]:
rsme_collabo = accuracy.rmse(test_pred,verbose=True)


## Make submission using test data


In [29]:
test = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv')

ratings=[]
for x,y in test.itertuples(index=False):
    output=svd.predict(x,y)
    ratings.append(output)
    
output_df=pd.DataFrame(ratings)[['uid','iid','est']]
output_df['ID']=output_df['uid'].astype(str) + '_' + output_df['iid'].astype(str)
output_df=output_df[['ID','est']]

results = pd.DataFrame({"Id":output_df['ID'],"rating": output_df['est']})
results.to_csv("SVD.csv", index=False)

In [30]:
my_submission = pd.DataFrame({'id': results.Id, 'rating': results.rating})
my_submission.to_csv('mark_myburgh.csv', index=False)